In [1]:
# first set global root data directory
import pylabs
pylabs.datadir.target = 'jaba'
from pathlib import *
import pandas as pd
import numpy as np
from pylabs.io.mixed import df2h5, h52df, get_h5_keys
from pylabs.utils import *
from pylabs.projects.genz.file_names import project, Optsd
import qgrid
qgrid.enable
import ipywidgets as widgets
fs = Path(getnetworkdataroot())
opts = Optsd()
style = {'description_width': 'initial'}
input_selkey = widgets.Dropdown(options=sorted(get_h5_keys(opts.info_fname, key='auto_qc')), disabled=False,
    description='Subject DWI to qc:', style=style)
input_selkey.layout.width = '350px'
input_selkey

/home/toddr/Software/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
setting root data directory to jaba.


Dropdown(description=u'Subject DWI to qc:', layout=Layout(width=u'350px'), options=('/sub-genz103/ses-1/dwi/auto_qc', '/sub-genz104/ses-1/dwi/auto_qc', '/sub-genz201/ses-1/dwi/auto_qc', '/sub-genz203/ses-1/dwi/auto_qc', '/sub-genz205/ses-1/dwi/auto_qc', '/sub-genz301/ses-1/dwi/auto_qc', '/sub-genz302/ses-1/dwi/auto_qc', '/sub-genz303/ses-1/dwi/auto_qc', '/sub-genz305/ses-1/dwi/auto_qc', '/sub-genz308/ses-1/dwi/auto_qc', '/sub-genz501/ses-1/dwi/auto_qc', '/sub-genz502/ses-1/dwi/auto_qc', '/sub-genz503/ses-1/dwi/auto_qc', '/sub-genz506/ses-1/dwi/auto_qc', '/sub-genz508/ses-1/dwi/auto_qc', '/sub-genz510/ses-1/dwi/auto_qc', '/sub-genz906/ses-1/dwi/auto_qc'), style=DescriptionStyle(description_width=u'initial'), value='/sub-genz103/ses-1/dwi/auto_qc')

In [2]:
pick = {'subj': input_selkey.value.split('/')[1], 'session': input_selkey.value.split('/')[2], }
# new h52df function makes all cols numeric. yes!
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
pick['dwi_fname'] = all_dwi_auto_qc.loc[0, 'dwi_fname']
# todo: make python scriptlet to load files into FSLeyes
# fslv_names = ' '.join([x + ' -b 1,200' for x in all_dwi_auto_qc.loc[0, ['dwi_fname', 'topup_fname', 'topdn_fname']].values])
# with WorkingContext(str(Path(all_dwi_auto_qc.loc[0, 'dwi_fname']).parent)):
#     with open('fslview_cmd.sh', 'w') as f:
#         f.write('fslview ' + fslv_names + ' &\n')

# limit to only active rows with mask
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] != -9999, 'itopup_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] != -9999, 'itopdn_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] == -9999, 'itopup_visqc'] = False
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] == -9999, 'itopdn_visqc'] = False
all_dwi_auto_qc.loc[:, 'dwi_visqc'] = True
with WorkingContext(str(Path(pick['dwi_fname']).parent/'qc')):  
    topup_gvolspng = open('{subj}_{session}_topup8b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topup_bvolspng = open('{subj}_{session}_topup8b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    topdn_gvolspng = open('{subj}_{session}_topdn7b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topdn_bvolspng = open('{subj}_{session}_topdn7b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi800_gvolspng = open('{subj}_{session}_b800-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi800_bvolspng= open('{subj}_{session}_b800-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi2000_gvolspng = open('{subj}_{session}_b2000-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi2000_bvolspng= open('{subj}_{session}_b2000-qc_bad_plot.png'.format(**pick), 'rb').read()

dwicols = [u'bvals', u'auto_dwi_vol_idx', u'auto_dwi_qc', 'dwi_visqc', u'x_bvec', u'y_bvec', u'z_bvec']
topudcols = [u'itopup', u'alltopup_idx', u'topup_qc', 'itopup_visqc', u'itopdn', u'topdn_qc', 'itopdn_visqc',]
dwi_auto_qc = all_dwi_auto_qc[dwicols]
topud_auto_qc = all_dwi_auto_qc[topudcols]
dwi_w = qgrid.show_grid(dwi_auto_qc, show_toolbar=False)
topupdn_w = qgrid.show_grid(topud_auto_qc, show_toolbar=False)

In [3]:
items = [widgets.Image(description='dwi b2000 bad vols', value=dwi2000_bvolspng,    format='png',    width=500,    height=600),
        widgets.Image(description='dwi b2000 good vols', value=dwi2000_gvolspng,    format='png',    width=500,    height=600),]
#items_layout = [widgets.Layout(flex-basis='50%', ), widgets.Layout(flex-basis='50%', widgets.Layout(flex-basis='100%',)]
box_layout = widgets.Layout(display='flex', align_items='stretch', border='solid', width='100%')
top_box = widgets.HBox(children=items)
bottom_box = widgets.HBox(children=[dwi_w])
dwi_box = widgets.Box([top_box, bottom_box], layout=box_layout)
dwi_box

Box(children=(HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xbf\xbf\xbf\x1f\x1f\x1f___???\xdf\xdf\xdf\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xf5Uk\xc9\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\x9d\xe9\x96\xa3*\x1bF?E\xb89\x7f\xf4\xaay\xae\xae\xae\xb3\xf4\xfe/\xe1\x93ytJ\x94H\xf2\xecuNWb\xd4W\x8c\xec\x00\x02\xfe\xaf\x07\x00\x80B\xf8\xdf\xa5\x0f\x00\x00\x00\x96\x02a\x01\x00\x8a\x01\xc2\x02\x00\x14\x03\x84\x05\x00(\x06\x08\x0b\x00P\x0c\x10\x16\x00\xa0\x18 ,\x00@1@X\x00\x80b\xb8\x19au\x1d\xb9\xf4!\\\x84NCHS\xcbEm\x17\xd2\x0cKI\xd7\xa5\xb6\xaf\x1aB\xf8\xb6U\xb0\xbcn\xe9\xb0\x9cF\xcb\x13\x8c\xec9\x17\xc4\x9e\x82\xf0p\xab.\xb8,\x12gkm\xac\x995XC\x16_\x89\xb7z\xcdN\x01am\x1a\xc3^\xf0zY\xcd/P\xea\xe5\x13\xb6O\x06N\xef\xd6\x17\x13\xe3\x8bV\x08\xabur{\xeb,\xaf\xa9]>\xab\xac\x9d\x84UIc2gQ\xf2\x14\x10/\xa9\xd4]_\xa4\xc2\x15S\xe2l\xadb6\xa9L\xecy\xe1\xde\x92\xd7l\x9c\xec\xf8\x12[\xb2\xa4P \xac\xed`\ta\xe9\x8cM\x99]\xaf\xda)\x03\x8f\t\xab\x11\x88#!\xfc0*\xb9\xa01\x1f\x89rG"\xaf1\xaa\xca%\x94\xf8i\xa8\xbc\xe5m\xb8]\xc0.\xc2\xaa\x8d\x87\x9c\xf0\xc9S@L2\xc5\xe1:\x97\x013\xb6\xd6$\xce\xd6*f\x93:\x9ct\xda.\xf5F\xe2\x9aM$;\xbe\xc4\x96,)\x15\x08k;j\x9e\x89\x15*\x17\xf0\x0b\xa5i\x1b/\x9f\xb0}J\x1c#\xbbu"W\xc3\x1a\xd4\xf9\xc8\xdf \xde\x9c\xef\xb1k\x99|\xdd:\x19\xb8\xe6\x97\xbe,\x98\x88\xa2\xd6L\xe5i\x8f\xf4\xd6\x9d<\xb3\xc45N\xfa\x14\xb8\x0b\xb9i+\xf7]\xe3_\x17\x13gk\x11\xb3I\x1dV`\x8b\xf7\x16_\xb3\x89d\xc7\x97\xd8\x92%\xc5\x02amG\x15e\xddJer^T\xd1\xbf\x88<#\xec \xac\xb1\xdd\x86\xaat\xca#s\xc2\xa2^\x19\x83\xd9\x0c\xec\xed\xa6\x99\xcd\xd7{\x08\x8b\xea\xd2Be\x859r\n\xbc\xf0\x95[\xa2\x1av\xc2\x85\xeb|i\x13gk\x11K\x84\xb5|o\xf15\x1b\';\xbe\xc4\x96,)\x17\x08k;\xda\xe8\xd7\x93\xe8\xdc\xc0TVjT\x89~\xe3\xc8\x13\xbb\xf5\xd2]{\xeff\x84U\x05u"\x9e\x81E\xe9\xa4\x8d\xaaQ\xeej1;\x08\xcbI\x88:\x9a\xf1S\xe0\x85g\x8e^\x19\xdf\x89g\xb0\xa9\xb3\xb5\x88\x9d\x85\x15\';\xbe\xc4\x16-)\x97\xeb\x11\x96j\x8cT?\x97\xcew-_\xf2\x7f\x19oC\xa6\xaa\x92\x13\xd0\xf2\x1f\xaf\xd6\xbd\xa2\xe4\x0eM\x05B| \xdb.M\x0c\x87^\x945\xfc}2\xafP\xae\x1a\x8aF\x85\xe5\' >\x80\xc4\x1a\xce\xb1-\x12V\xef\x95\'f\x84E\xc2\x02\xa3\xce.A\xb5\xa6qkY\xa9\x03\x15{\x96\x8b\xc2\xc4\xb5f\x17\xf2\x93\xca\xd8\xafU\x85\x81t\x9a[[TP\'y\xfc\x14\x8c\nK\xee\xc4\xdbf\xe2lyAM\xa2\xbdd8\xb1\xd2\x87\xdd\xb9\x07\xe9o*O\x12\xf1Net\xcd\xc6\xc9\x8e/\xb1%K<T\x84\x89K\xffH\\\x8b\xb0lc\xa4\xfc\xbdL\tK\xafC\xe2\x1co>2\xcd L\xefP\x972\xf8\x07m\x10\xc3\x17\x16\r\x7f\x10+\xa7"(_\xd6\xd5@\xf2W6L@|\x00\xd1\x1a\xce\xc6\xa3\xbb\r\xb2`\xe5\xd6\t\xa6\x85U\xc7U=\x99\x81\xeb\xe0\x00X]\xb3\xc9\xa4\xf0=\xd3\xb1\xc4\xe9z\x8d<,\xde\xccR\xb9\xb1F\xd2\xdc8*\x91\x07>~\n\xbc\x85\xad\x93c\xa5x=\xdfN\x9c-/\xe3+\xcd\x05\xc90\xb1\xc6\x0e\xdb\xb9X\xc2M\xf5\xe5\x15\x08\xcb\xbff\xe3d\xc7\x97\xd8\x92%.S\x97\xfe)=;v\xe6J\x84\xc5\xcf1iZ^L\x92\xdfHJX\xe27O\xe4\x1c\x16n\xce\xbf\x1d\xca\xdb2\xa9\xfa\xd6\xd4\x0eEE\xc3\\Q\xd5\

In [4]:
items = [widgets.Image(description='topup bad vols', value=topup_bvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topup good vols', value=topup_gvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn bad vols', value=topdn_bvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn good vols', value=topdn_gvolspng,    format='png',    width=400,    height=600),
]
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
topupdn_box = widgets.Box(children=items+[topupdn_w], layout=box_layout)
topupdn_box

Box(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xdf\xdf\xdf___???\xbf\xbf\xbf\x1f\x1f\x1f\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xa8*\xe1\x90\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\xdd\t\x96\xa2J\xb7\x06\xd0G\x17\x93c\x18\xff\xc2\xf9\x0f\xe1I\x13H\xa7)U&yO\xd6\xdek\xdd[\xa9"\x87@\xf8\x84\xa0\xf1\xffn\x00A\xfc\xdfOO\x00\xc0\xbb\x04\x16\x10\x86\xc0\x02\xc2\x10X@\x18\x02\x0b\x08C`\x01a\x08, \x0c\x81\x05\x84\xf1\x8b\x03\xab\xeb\xd2OO\xc2\x8f\xe8\xb2\x94\xaaf|\xaa\xed\xb6\xaa\xfb\xb3\xa9\xeb\x8e\xde_T)\xf5\xef-6\xcf7m}\x7f\xbe\xde=\x7f\xe0\xc9\x98\xbfKz4y;yE\xb7^\x0c\x0ef\xce\xd9R_\x0cQViU\xf1\x87\xfd\xb6\xb5@`\xfde\x8d\xc7\x1a\x90\x9fk\xfa%\xb6^\xad7\xe5\xf7\xac\xc0\xc7\xa3]\x07S\xd9?u"\xb0\xda\xc5\xda\xdf.\x9eo\xea\xc7\xf3_F\xd67\x05\xd6\xc1\x9c\xcd\xe5\x16\xear\xf1\xd20\xd5\x8bd:\x989\xa7|\xd9\xb2r\x18\xf3\x9bck\xab\xea\xf4\x14|\xa5\xa9\xea>\xb7s\x9b\x8f\xd6\x82g\xf31\x02\x81\xf57\xca\x83\xc0\xaa\x0f\xd6\x9b\xe2{V\xe0\xe3\xd1\xf6k\xe2`\x98\x92\xd4OF1>Q\xcd/\r\xdb!\x07+_YO\xdb)uZ\xb7\xa1X=\xdfn\xdf\xb7\xf1=\x81u4gs\xb9\xa9Y\xc3\xe4\xa5\xc7+\xe5\x9c\xce\x93\x83\x99s\xca\x97-\xbb\xcf\xe3\xba}7\x0b>?\x9f\xca\xf9{e\x9aK\x07k\xc1\xd3\xf9\x18\x81\xc0\xfa\x1bM\xbf\x12O\xa6\xb5\xa2_\x1a\xaa\xb6Z\xad7\xe5\xf7\xac\xc0OF\xbb\xa8\\\xdc\x87\xa8\x17/\xad\xdf\xb0\x7f{?\xc6\xae\x1d\x97\xe3\xb2]\xac\xd1M\xbf|\x8f_\xda\xc3\xa6\xd6\x17{S\xdf\xd2\xde\xa39\xbb/\xd7\'k\xb1|T\xad\x86\x7f1s\xde\xf2e\xcb\xee\x03\xbc\x9f\x03\x1f\x9fO\xc3\x07x\x9fI\xfd\xbc\x1a\xdb\xb6\x9f[\xcf\xe7c\x04\x02\xebo\x14\xbbU\xb7\x98V\xf2\xfe\x9b.o\x87\xf4k\xc67\xac\xc0\xcfF\xbb\x8d\xca\xc5\xe6\xd0W\x81U\xaf6:\xca\xc7\x1a\xbd\x1aM\xf5\xe5\x8a\xfe\x1d\x81u4g\x8f\xca\x15\xcb-\xaa\xfb\xb0}\xc0>>\xa4\x173\xe7-\xef\x04\xd6\x07\xc7vV\x95\xb7\x9b\xe6\xb6\xed\xd6\x82\x17\xf31\x02\x81\xf57\xda\xdd\xd7i\xcakG9\xa5I5u\xb0|\xb8\xf2\x8b\xd1\xae\xda\xdd\xac\x1e}\x11XE\xb7\xdeI\xea\x97\xfaak\xa5]\xedV\xf5\xfbU\xaf\xb7"\xbe#\xb0vs\xf6I\xb9r\x11\xa7e\xdf\xfaU\x82\xbd\x989\xefN\xc5_\x0e\xf0\xc7\x03\xbf9\xc2\xc76\xf10\x1bvM|1\x1f#\x88\x1cX\xc5x\xd4j\xfa\xfa\\|2\xe3\x9f\xfd\xff\xcb\xbe\x0f\xb9n\x0fW\xaf\xb6\xee_Z.5\xe3\x08\xe7\x1d\x8a\xe1\x85\xb1\x83r\xae\xb1p\x1b\xbe\xd0\xd6\xe3,W}YSG\xd1\xd3\xc0Z7`?\x01\x07C,\xa6\xed\xad\xc0\xba\xad\xb6/\xbe\x08\xac\xd4m6\x18\xf3\x1a\xbd\xd9\xcf\xa9\x96{]G\x13:\x8cy|j\xdb\xb8v\x1e\xc5\xf8J1\xa7_;}\xe3\x1f\xb7y?g\x9f4d\x19X\xe3(\x97\xb3\xe9\xc5\xccyL\xc1\xb2\x8d\xab\xa9^\x94:\x9e\xcan\xf9\xb1\x1c\xbd\xf5\xbeY\\\x1c\x0e\xbc\x1dz\x98\xd0q\x11-w\x93>\xfd\xf9j\x98r\x11X\xcb\xb5\xe0\xd5|\x1c^\x1f\x87}\xb1Z\xfc\xac\xb8\x81\xd5\xcc\xc7\x86\xc6\xef\xcf\xa3\xc0\xca\xc3\xa4\xfd\x1a?\xbf4\xf7\x04\x95y\x84y+\xa3\x7f\xa1\xdd\xd4X\x07V\xfd\xa89*\x16;\x82\xe3\x9fMqw\xf8M\xbam\xc0~\x02vC,\xde\xfct\xb4\x9bu\xb2Xn\xf8\xbf\x0e\xacf\xbf\xab7\xae\xd1\xcdf\x02\xca\xa6Y\x7f\x05l\'\xb4\x1fs\xfd\xacq\xcdj\xb2\xfa\xbe\x94bY\xebI\x9b\xf7s\xf6IC\xda\xc5j

In [6]:
dwi_w.get_changed_df()
topupdn_w.get_changed_df()
all_dwi_visqc = pd.concat([dwi_w.get_changed_df(), topupdn_w.get_changed_df()], axis=1)
df2h5(all_dwi_visqc, opts.info_fname, input_selkey.value.replace('auto_qc', 'vis_qc'))

In [5]:
topupdn_w.get_changed_df()

,itopup,alltopup_idx,topup_qc,itopup_visqc,itopdn,topdn_qc,itopdn_visqc
0,0,0,0,True,0,0,True
1,1,1,0,True,1,0,True
2,2,2,0,True,2,0,True
3,3,3,0,True,3,0,True
4,4,4,0,True,4,0,True
5,5,5,0,True,5,0,True
6,6,6,0,True,6,0,True
7,-9999,7,0,False,-9999,-9999,False
8,-9999,-9999,-9999,False,-9999,-9999,False
9,-9999,-9999,-9999,False,-9999,-9999,False


In [7]:
dwi_w.get_changed_df()

,bvals,auto_dwi_vol_idx,auto_dwi_qc,dwi_visqc,x_bvec,y_bvec,z_bvec
0,0,0,0,True,0.000,0.000,0.000
1,2000,0,1,True,1.000,0.000,0.000
2,2000,1,0,True,0.000,1.000,0.000
3,2000,2,0,True,0.000,0.000,1.000
4,2000,3,0,True,-0.625,-0.777,0.076
5,2000,4,0,True,-0.745,0.623,0.237
6,2000,5,1,True,0.025,-0.095,0.995
7,2000,6,0,True,-0.853,-0.004,-0.522
8,2000,7,1,True,-0.155,0.904,-0.399
9,2000,8,1,True,-0.025,-0.716,-0.698
